In [1]:
import pandas as pd
import json
import math
import os
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn_pandas import DataFrameMapper
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
dataset_home = "datasets/haskell"    

exercises_df = pd.DataFrame(
    columns=['exercise.name', 'submission_amount', 'aborted', 'errored', 'failed', 'passed',
             'passed_with_warnings', 'pending', 'manual_evaluation_pending'])

for exercise in os.listdir(dataset_home):
    json_file = open(os.path.join(dataset_home, exercise), encoding='utf8')
    json_data = json.load(json_file)
    submissions = pd.io.json.json_normalize(json_data)
    total_amount_submissions = submissions.shape[0]
    submissions_grouped = submissions.groupby(['status']).size()
    metrics = {}
    metrics = submissions_grouped
    metrics['exercise.name'] = str(exercise)
    metrics['submission_amount'] = total_amount_submissions
    exercises_df = exercises_df.append(metrics, ignore_index=True)


In [ ]:
submissions.columns

In [3]:
exercises_df[exercises_df.submission_amount >100].sort_values('submission_amount').tail(10)


,exercise.name,submission_amount,aborted,errored,failed,passed,passed_with_warnings,pending,manual_evaluation_pending,running
31,submissions_Tipando más funciones.json,6328,53,1703,3442,1023,107,NaN,NaN,NaN
194,submissions_Puntos Para Setenta!.json,6450,78,1553,3614,574,631,NaN,NaN,NaN
152,submissions_Para cerrar....json,8193,96,3715,2513,568,1300,1,NaN,NaN
80,submissions_haceFrioF.json,10367,184,3405,3534,1361,1883,NaN,NaN,NaN
7,submissions_Tipos complicados.json,10463,147,1947,7242,615,509,3,NaN,NaN
253,submissions_esMultiploDe.json,11698,130,4123,4276,2303,865,1,NaN,NaN
272,submissions_Más clases de tipos.json,11964,110,4635,6222,993,4,NaN,NaN,NaN
50,submissions_esMultiploDeTres.json,12956,175,6041,4886,1830,24,NaN,NaN,NaN
149,submissions_Restricciones Múltiples.json,13037,121,2664,9192,821,239,NaN,NaN,NaN
257,submissions_esBisiesto.json,22751,329,5416,14020,1925,1057,4,NaN,NaN


We choosed mitad exercise beacause the submissions are distributed as we want

# Load Submissions of Mitad exercise
Exercise link:
https://central.mumuki.io/exercises/4256-programacion-funcional-practica-valores-y-funciones-mitad#editor

In [31]:
dataset_home = "datasets/haskell"
json_file = open(os.path.join(dataset_home, 'submissions_mitad.json'), encoding='utf8')
json_data = json.load(json_file)
submissions = pd.io.json.json_normalize(json_data)
total_amount_submissions = submissions.shape[0]
submissions_grouped = submissions.groupby(['status']).size()
metrics = {}
metrics = submissions_grouped
metrics['submission_amount'] = total_amount_submissions
metrics/total_amount_submissions

status
aborted                 0.011635
errored                 0.101222
failed                  0.190809
passed                  0.695172
passed_with_warnings    0.001163
submission_amount       1.000000
dtype: float64

# Narrowing Dataset

In [4]:
submissions = submissions[submissions['content'] != ""]
submissions = submissions[~submissions['content'].isnull()]
submissions = submissions[~(submissions['status'] == 'aborted')]
submissions = submissions[~(submissions['status'] == 'pending')]
#Improve it
submissions['result'] =  submissions.apply(
    lambda x : str(x.test_results) if x.status == 'failed' else (
        str(x.expectation_results) if x.status == 'passed_with_warnings' else('passed' if x.status == 'passed' else x.result)), axis=1)

As we choose only one exercise, in train set we only need sumbmission's content

In [7]:
X = submissions['content']
Y = submissions['status']

In [8]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.33, random_state=42)

# Size of datasets
* X train 1131 
* X test 558 
* Y train 1131
* Y test 558

In [9]:
le = LabelEncoder()
le.fit(Y.unique())

LabelEncoder()

In [27]:
cv = CountVectorizer()
cv.fit(X_train)

cv.vocabulary_

{'mitad': 24,
 'numero': 42,
 'float': 12,
 'div': 4,
 'num': 39,
 'mod': 29,
 'valor': 51,
 'abs': 1,
 'nro': 38,
 'con': 2,
 'notacion': 37,
 'infija': 20,
 'point': 46,
 'free': 15,
 'double': 8,
 'nunero': 44,
 'mtad': 32,
 'num1': 40,
 'divide': 5,
 'flip': 11,
 'mitadnumero': 26,
 'mitadn': 25,
 'n1': 34,
 'n2': 35,
 'decimalpart': 3,
 'int': 21,
 'fromintegral': 16,
 'foo': 13,
 'modn': 31,
 'unnro': 49,
 'division': 7,
 'mitado': 27,
 'rem': 48,
 'munero': 33,
 'mod2': 30,
 'la': 23,
 'funcion': 17,
 'es': 10,
 'mitas': 28,
 'no': 36,
 'funciona': 18,
 'porque': 47,
 'para': 45,
 'enteros': 9,
 'numero2': 43,
 'unnumero': 50,
 'integer': 22,
 'number': 41,
 'half': 19,
 'dividir': 6,
 '10': 0,
 'fractional': 14}

In [11]:
X_train_trans = cv.transform(X_train)
Y_train_trans = le.transform(Y_train)

In [12]:
lr = LinearRegression()
lr.fit(X_train_trans, Y_train_trans)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [13]:
X_test_trans = cv.transform(X_test)
Y_test_trans = le.transform(Y_test)

In [14]:
Y_predicted = lr.predict(X_test_trans)

# Metrics

In [15]:
print("Mean squared error: %.2f"
      % mean_squared_error(Y_test_trans, Y_predicted))
print('Variance score: %.2f' % r2_score(Y_test_trans, Y_predicted))

Mean squared error: 0.35
Variance score: 0.22


# Mini example

In [16]:
my_subm = ['mitad x = x /2']
my_transf = cv.transform(my_subm)

In [17]:
predicted = lr.predict(my_transf)

In [18]:
predicted[0]

1.64473895313893

In [19]:
le.inverse_transform(math.ceil(predicted[0]))

/home/mrc/anaconda3/envs/mumuki/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


'passed'